<a href="https://colab.research.google.com/github/thiagoudweb/Projeto-RAg---1a-VA/blob/main/RAGIPOB3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain langchain-community langchain-openai langchain-groq
!pip install streamlit streamlit_chat
!pip install sentence-transformers
!pip install chromadb
!pip install pypdf unstructured
!pip install PyPDF2



In [15]:
import langchain
import streamlit as st
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
import os
import PyPDF2
from langchain.text_splitter import RecursiveCharacterTextSplitter



In [ ]:
import PyPDF2

def carregar_texto_pdf(caminho_pdf):
    texto_completo = ""
    with open(caminho_pdf, "rb") as arquivo:
        leitor = PyPDF2.PdfReader(arquivo)
        for pagina in range(len(leitor.pages)):
            texto_completo += leitor.pages[pagina].extract_text() + "\n"
    return texto_completo


In [21]:
def dividir_texto(texto, tamanho_chunk=500, sobreposicao=100):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=tamanho_chunk,
        chunk_overlap=sobreposicao,
        separators=["\n\n", "\n", " ", ""]
    )
    return splitter.split_text(texto)
